In [1]:
def grep(pattern):
   print("start grep for", pattern)
   while True:
       s = yield
       if pattern in s:
           print("found!", s)
       else:
           print("no %s in %s" % (pattern, s))

In [36]:
gr = grep("python")

In [3]:
gr

<generator object grep at 0x122c2a2f0>

In [4]:
next(gr)

start grep for python


In [5]:
gr.send("")

no python in 


In [6]:
gr.send("python")

found! python


In [7]:
next(gr)

TypeError: argument of type 'NoneType' is not a container or iterable

In [8]:
gr.send("python")

StopIteration: 

In [37]:
gr = grep("python")

In [38]:
gr.send(None)

start grep for python


In [39]:
gr.send("wdwd")

no python in wdwd


In [40]:
gr.throw(ValueError("wrong"))

ValueError: wrong

In [10]:
lst = [1, 2, 3, 4, 5]

In [11]:
it1 = iter(lst)

In [12]:
it1

In [13]:
next(it1)

1

In [14]:
next(it1)

2

In [15]:
it2 = iter(it1)

In [16]:
it2

In [17]:
it1 is it2

True

In [18]:
next(it2)

3

In [19]:
it3 = iter(lst)

In [20]:
type(it1)

list_iterator

In [21]:
it3 is it1

False

In [22]:
next(it3), next(it3)

(1, 2)

In [23]:
next(it1)

4

In [24]:
class SpecialIterator:
    def __init__(self, limit):
        self.limit = limit
        self.counter = 0

    def __iter__(self):
        print("iter")
        return self
        
    def __next__(self):
        print("next")
        if self.counter < self.limit:
            self.counter += 1
            return self.counter
        else:
            raise StopIteration

In [28]:
it1 = SpecialIterator(5)

In [29]:
next(it1), next(it1)

next
next


(1, 2)

In [30]:
it2 = iter(it1)

iter


In [31]:
it2 is it1, next(it2)

next


(True, 3)

In [33]:
next(it2)

next


4

In [34]:
next(it2)

next


5

In [35]:
next(it2)

next


StopIteration: 

In [32]:
for i in SpecialIterator(3):
    print(f"{i=}")

iter
next
i=1
next
i=2
next
i=3
next


In [41]:
it = iter(range(10))

result = list(
    zip(
        it,
        map(lambda x: x ** 2, it)
    )
)
print(result)

[(0, 1), (2, 9), (4, 25), (6, 49), (8, 81)]


In [43]:
# start..stop step, step - update

def counter(start, stop, step):
    while start < stop:
        step_candidate = yield start
        if step_candidate is not None and isinstance(step_candidate, int):
            step = step_candidate

        start += step


cnt = counter(-5, 10, 3)
assert next(cnt) == -5
assert next(cnt) == -2
assert cnt.send(5) == 3
assert next(cnt) == 8
assert cnt.send(1) == 9
print("ok")

ok


In [68]:
import asyncio
import time
import aiohttp

In [47]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    await say_after(1, "hello")
    await say_after(2, "world")
    print(f"finished at {time.strftime('%X')}")


await main()

started at 20:25:16
hello
world
finished at 20:25:19


In [48]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    say1 = say_after(1, "hello")
    await say_after(2, "world")
    await say1

    print(f"finished at {time.strftime('%X')}")


await main()

started at 20:26:45
world
hello
finished at 20:26:48


In [49]:
coro = main()

In [50]:
coro

<coroutine object main at 0x123201460>

In [52]:
await coro

started at 20:27:44
world
hello
finished at 20:27:47


In [53]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    
    await asyncio.create_task(say_after(1, "hello"))
    await asyncio.create_task(say_after(2, "world"))
    
    print(f"finished at {time.strftime('%X')}")


await main()

started at 20:29:06
hello
world
finished at 20:29:09


In [58]:
def async_timeit(coro):
    async def inner(*args, **kwargs):
        t1 = time.time()
        try:
            return await coro(*args, **kwargs)
        finally:
            t2 = time.time()
            print(f"Total time `{coro.__name__}`", t2 - t1)

    return inner

In [59]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


@async_timeit
async def main():
    print(f"started at {time.strftime('%X')}")
    
    task1 = asyncio.create_task(say_after(1, "hello"))
    await asyncio.create_task(say_after(2, "world"))
    await task1
    
    print(f"finished at {time.strftime('%X')}")


await main()

started at 20:35:37
hello
world
finished at 20:35:39
Total time `main` 2.0020670890808105


In [60]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


@async_timeit
async def main():
    print(f"started at {time.strftime('%X')}")
    
    task1 = asyncio.create_task(say_after(1, "hello"))
    task2 = asyncio.create_task(say_after(2, "world"))

    await asyncio.gather(task1, task2)
    
    print(f"finished at {time.strftime('%X')}")


await main()

started at 20:37:09
hello
world
finished at 20:37:11
Total time `main` 2.002526044845581


In [61]:
async def say_after(delay, what):
    time.sleep(delay)
    print(what)


@async_timeit
async def main():
    print(f"started at {time.strftime('%X')}")
    
    task1 = asyncio.create_task(say_after(1, "hello"))
    task2 = asyncio.create_task(say_after(2, "world"))

    await asyncio.gather(task1, task2)
    
    print(f"finished at {time.strftime('%X')}")


await main()

started at 20:37:52
hello
world
finished at 20:37:55
Total time `main` 3.009917974472046


In [62]:
async def say_after(delay, what):
    #time.sleep(delay)
    await asyncio.to_thread(time.sleep, delay)
    print(what)


@async_timeit
async def main():
    print(f"started at {time.strftime('%X')}")
    
    task1 = asyncio.create_task(say_after(1, "hello"))
    task2 = asyncio.create_task(say_after(2, "world"))

    await asyncio.gather(task1, task2)
    
    print(f"finished at {time.strftime('%X')}")


await main()

started at 20:42:15
hello
world
finished at 20:42:17
Total time `main` 2.006805181503296


In [67]:
async def say_after(delay, what):
    for i in range(10):
        print("inner", what)
        time.sleep(delay / 10)
        await asyncio.sleep(0)
    print(what)


@async_timeit
async def main():
    print(f"started at {time.strftime('%X')}")
    
    task1 = asyncio.create_task(say_after(1, "hello"))
    task2 = asyncio.create_task(say_after(2, "world"))

    await asyncio.gather(task1, task2)
    
    print(f"finished at {time.strftime('%X')}")


await main()

started at 20:46:18
inner hello
inner world
inner hello
inner world
inner hello
inner world
inner hello
inner world
inner hello
inner world
inner hello
inner world
inner hello
inner world
inner hello
inner world
inner hello
inner world
inner hello
inner world
hello
world
finished at 20:46:21
Total time `main` 3.0893819332122803


In [82]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 20


async def fetch_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            return await resp.text()


async def fetch_batch_urls(urls):
    for url in urls:
        await fetch_url(url)


@async_timeit
async def run():
    await fetch_batch_urls(URLS)


await run()

Total time `run` 7.781434774398804


In [83]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 20


async def fetch_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            return await resp.text()


async def fetch_batch_urls(urls):
    tasks = [
        asyncio.create_task(fetch_url(url))
        for url in urls
    ]
    await asyncio.gather(*tasks)


@async_timeit
async def run():
    await fetch_batch_urls(URLS)


await run()

Total time `run` 7.566035270690918


In [84]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 20


async def fetch_url(url):
    await asyncio.sleep(1)


async def fetch_batch_urls(urls):
    for url in urls:
        await fetch_url(url)


@async_timeit
async def run():
    await fetch_batch_urls(URLS)


await run()

Total time `run` 20.021975994110107


In [85]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 20


async def fetch_url(url):
    await asyncio.sleep(1)


async def fetch_batch_urls(urls):
    tasks = [
        asyncio.create_task(fetch_url(url))
        for url in urls
    ]
    await asyncio.gather(*tasks)


@async_timeit
async def run():
    await fetch_batch_urls(URLS)


await run()

Total time `run` 1.0017168521881104


In [88]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 20
WORKERS = 4


async def fetch_url(url):
    await asyncio.sleep(1)


async def fetch_worker(que, name):
    print("fetch_worker started", name)
    cnt = 0
    while True:
        url = await que.get()
        if url is None:
            await que.put(url)
            break

        try:
            result = await fetch_url(url)
        except Exception:
            pass
        cnt += 1
    
    print("fetch_worker finished", name, cnt)


@async_timeit
async def run():
    que = asyncio.Queue()
    for url in URLS:
        await que.put(url)
    await que.put(None)
        
    tasks = [
        asyncio.create_task(fetch_worker(que, f"fetcher_{i}"))
        for i in range(WORKERS)
    ]
    await asyncio.gather(*tasks)


await run()

fetch_worker started fetcher_0
fetch_worker started fetcher_1
fetch_worker started fetcher_2
fetch_worker started fetcher_3
fetch_worker finished fetcher_0 5
fetch_worker finished fetcher_1 5
fetch_worker finished fetcher_2 5
fetch_worker finished fetcher_3 5
Total time `run` 5.005489826202393


In [90]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 20
WORKERS = 4


async def fetch_url(url, session):
    async with session.get(url) as resp:
        return await resp.text()


async def fetch_worker(que, session, name):
    print("fetch_worker started", name)
    cnt = 0
    while True:
        url = await que.get()
        if url is None:
            await que.put(url)
            break

        try:
            result = await fetch_url(url, session)
        except Exception:
            pass
        cnt += 1
    
    print("fetch_worker finished", name, cnt)


@async_timeit
async def run():
    que = asyncio.Queue()
    for url in URLS:
        await que.put(url)
    await que.put(None)

    async with aiohttp.ClientSession() as session:
        tasks = [
            asyncio.create_task(fetch_worker(que, session, f"fetcher_{i}"))
            for i in range(WORKERS)
        ]
        await asyncio.gather(*tasks)


await run()

fetch_worker started fetcher_0
fetch_worker started fetcher_1
fetch_worker started fetcher_2
fetch_worker started fetcher_3
fetch_worker finished fetcher_0 4
fetch_worker finished fetcher_2 6
fetch_worker finished fetcher_1 6
fetch_worker finished fetcher_3 4
Total time `run` 1.5025699138641357
